In [2]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
# %qtconsole

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from IPython.display import display
def display_all(df):
    with pd.option_context("display.max_rows", 1000): 
        with pd.option_context("display.max_columns", 1000): 
            display(df)
			
import pandas as pd
import numpy as np
from pathlib import Path		

import warnings
warnings.filterwarnings('ignore')
# warnings.filterwarnings(action='once')

C:\Users\KML\Anaconda3\lib\importlib\_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [64]:
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

In [65]:
df = pd.read_csv(r'data/merged_date_v2.csv', parse_dates = ['count_date'])
df.head()

,count_date,road_name,adt,latitude,longitude,sampling_count
0,1974-07-12,COLWILL RD,125.0,-36.832191,174.636777,3
1,1974-08-11,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22
2,1974-08-11,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1
3,1974-08-11,TAIKATA RD,2490.0,-36.841192,174.651345,1
4,1974-08-11,GLORIA AVE,1823.0,-36.848881,174.650064,5


In [66]:
import re
def add_datepart(df, fldname, drop=True, time=False, errors="raise"):	

    fld = df[fldname]
    fld_dtype = fld.dtype
    if isinstance(fld_dtype, pd.core.dtypes.dtypes.DatetimeTZDtype):
        fld_dtype = np.datetime64

    if not np.issubdtype(fld_dtype, np.datetime64):
        df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True, errors=errors)
    targ_pre = re.sub('[Dd]ate$', '', fldname)
    attr = ['Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
            'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start']
    if time: attr = attr + ['Hour', 'Minute', 'Second']
    for n in attr: df[targ_pre + n] = getattr(fld.dt, n.lower())
    df[targ_pre + 'Elapsed'] = fld.astype(np.int64) // 10 ** 9
    if drop: df.drop(fldname, axis=1, inplace=True)

In [67]:
# add date parts
# df.reset_index(inplace = True)
add_datepart(df, 'count_date')

In [68]:
df['road_name_code'] = df['road_name'].astype('category').cat.codes

In [69]:
df.columns

Index(['road_name', 'adt', 'latitude', 'longitude', 'sampling_count',
       'count_Year', 'count_Month', 'count_Week', 'count_Day',
       'count_Dayofweek', 'count_Dayofyear', 'count_Is_month_end',
       'count_Is_month_start', 'count_Is_quarter_end',
       'count_Is_quarter_start', 'count_Is_year_end', 'count_Is_year_start',
       'count_Elapsed', 'road_name_code'],
      dtype='object')

In [19]:
df.head()

,road_name,adt,latitude,longitude,sampling_count,count_Year,count_Month,count_Week,count_Day,count_Dayofweek,count_Dayofyear,count_Is_month_end,count_Is_month_start,count_Is_quarter_end,count_Is_quarter_start,count_Is_year_end,count_Is_year_start,count_Elapsed
0,774,125.0,-36.832191,174.636777,3,1974,7,28,12,4,193,False,False,False,False,False,False,142819200
1,1695,1900.0,-36.803623,174.638971,22,1974,8,32,11,6,223,False,False,False,False,False,False,145411200
2,1695,2853.0,-36.797016,174.650038,1,1974,8,32,11,6,223,False,False,False,False,False,False,145411200
3,3628,2490.0,-36.841192,174.651345,1,1974,8,32,11,6,223,False,False,False,False,False,False,145411200
4,1365,1823.0,-36.848881,174.650064,5,1974,8,32,11,6,223,False,False,False,False,False,False,145411200


In [70]:
xcols = ['road_name_code', 'latitude', 'longitude', 
       'count_Year', 'count_Month', 'count_Week', 'count_Day',
       'count_Dayofweek', 'count_Dayofyear', 'count_Is_month_end',
       'count_Is_month_start', 'count_Is_quarter_end',
       'count_Is_quarter_start', 'count_Is_year_end', 'count_Is_year_start',
       'count_Elapsed']
ycol = 'adt'

In [71]:
# train test split
from sklearn.model_selection import RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
import math

In [82]:
def rmse(x,y): 
    # calculates r^2
    return math.sqrt(np.square(np.subtract(x,y)).mean())
def get_scores(m, X_train, X_valid, y_train, y_valid):
    # returns rmsq training, r^2 validation, avg accuracy training, avg accuracy validation, oob
    res = [rmse(m.predict(X_train), y_train), rmse(m.predict(X_valid), y_valid),
                m.score(X_train, y_train), m.score(X_valid, y_valid)]
    if hasattr(m, 'oob_score_'): 
        res.append(m.oob_score_)
    return res

def KFold_fit_regression_fit(model, X, y, splits = 5, repeats = 10, random_state = None):
    folds = RepeatedKFold(n_splits = splits, n_repeats = repeats, random_state = random_state)
#     predicted_solution = []
#     correct_solution = []
    results = []
    for train_ind, test_ind in folds.split(X, y):
        X_train, X_test = X.iloc[train_ind,:], X.iloc[test_ind,:]
        y_train, y_test = y[train_ind], y[test_ind]
        model.fit(X_train, y_train)
        r = get_scores(model,X_train, X_test, y_train, y_test)
#         predicted_solution += list(model.predict(X_test))
#         correct_solution += list(y_test)
        results.append(r)
    return np.vstack(results)


# Kfold

In [46]:
m = RandomForestRegressor(n_estimators=500, oob_score= True)
m

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=None,
           oob_score=True, random_state=None, verbose=0, warm_start=False)

In [41]:
X = df[xcols]
y = df[ycol]
results = KFold_fit_regression_fit(m, X, y, splits = 5, repeats=1, random_state=100 )
results

[[1468.967855146438,
  4102.878027104442,
  0.9671035172933449,
  0.743212704719542,
  0.757088344510431],
 [1436.6949050331023,
  3872.3652364781965,
  0.9684402018118787,
  0.7739189153874679,
  0.7670426104085492],
 [1430.0484511688526,
  3857.0724308684844,
  0.9690531319618426,
  0.766113702773024,
  0.7711441437404165],
 [1440.6966708913965,
  3981.2006201591375,
  0.9684309986751617,
  0.7559464541471671,
  0.7672302276024793],
 [1463.0467730107875,
  3909.934537201361,
  0.967124052087147,
  0.77350651951373,
  0.7577262270038259]]

In [ ]:
x = np.array([1,2,3,4,5])
y = np.array([2,3,4,5,6])


In [49]:
r = np.vstack(results)
# r = r.mean(axis = 0)
r.mean(axis = 0)

array([1.44789093e+03, 3.94469017e+03, 9.68030380e-01, 7.62539659e-01,
       7.64046311e-01])

# Use oob for model selection and parameter tuning

In [73]:
df[:10]

,road_name,adt,latitude,longitude,sampling_count,count_Year,count_Month,count_Week,count_Day,count_Dayofweek,count_Dayofyear,count_Is_month_end,count_Is_month_start,count_Is_quarter_end,count_Is_quarter_start,count_Is_year_end,count_Is_year_start,count_Elapsed,road_name_code
0,COLWILL RD,125.0,-36.832191,174.636777,3,1974,7,28,12,4,193,False,False,False,False,False,False,142819200,774
1,HOBSONVILLE RD,1900.0,-36.803623,174.638971,22,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,1695
2,HOBSONVILLE RD,2853.0,-36.797016,174.650038,1,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,1695
3,TAIKATA RD,2490.0,-36.841192,174.651345,1,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,3628
4,GLORIA AVE,1823.0,-36.848881,174.650064,5,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,1365
5,HOBSONVILLE RD,2163.0,-36.814339,174.629872,2,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,1695
6,TE ATATU RD,3733.0,-36.832667,174.648011,11,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,3695
7,KOTUKU ST,1700.0,-36.850907,174.648874,5,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,2029
8,HOBSONVILLE RD,3900.0,-36.818111,174.615714,20,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,1695
9,TE ATATU RD,14450.0,-36.856203,174.651780,10,1974,8,32,11,6,223,False,False,False,False,False,False,145411200,3695


In [84]:
from sklearn.utils import shuffle
def split_vals(df, n):
    return df[:n].copy(), df[n:].copy()
train_size = int(df.shape[0]*0.75)
df_model = shuffle(df)
X, y = df_model[xcols], df_model[ycol]
X_train, X_test = split_vals(X, train_size)
y_train, y_test = split_vals(y, train_size)

X_train.head()

,road_name_code,latitude,longitude,count_Year,count_Month,count_Week,count_Day,count_Dayofweek,count_Dayofyear,count_Is_month_end,count_Is_month_start,count_Is_quarter_end,count_Is_quarter_start,count_Is_year_end,count_Is_year_start,count_Elapsed
11304,2097,-36.824130,174.611905,2015,3,13,24,1,83,False,False,False,False,False,False,1427155200
9479,2666,-36.711192,174.778044,2013,7,27,3,2,184,False,False,False,False,False,False,1372809600
1591,523,-36.903297,174.621134,1985,6,24,15,5,166,False,False,False,False,False,False,487641600
17963,2447,-36.888823,174.633541,2018,3,9,1,3,60,False,True,False,False,False,False,1519862400
10037,3931,-36.614636,174.714830,2013,12,49,4,2,338,False,False,False,False,False,False,1386115200


In [109]:
m = RandomForestRegressor (n_estimators=150, max_depth=38, min_samples_leaf = 3, max_features=0.9, oob_score= True, random_state=1, n_jobs  = 3)
m.fit(X_train, y_train)
m.oob_score_
m.score(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=38,
           max_features=0.9, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=150, n_jobs=3,
           oob_score=True, random_state=1, verbose=0, warm_start=False)

0.7304259274406519

0.9099333969704781

In [92]:
m.oob_score_
m.score(X_train, y_train)

0.7345326081051224

0.9639209597647438

In [111]:
def rf_feat_importance(m, df):
    return pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
imp = rf_feat_importance(m, X_train)
imp

,cols,imp
2,longitude,0.281669
1,latitude,0.245266
0,road_name_code,0.179227
15,count_Elapsed,0.140939
8,count_Dayofyear,0.040654
7,count_Dayofweek,0.037727
6,count_Day,0.033203
5,count_Week,0.018571
3,count_Year,0.015339
4,count_Month,0.006086


In [114]:
cols_to_drop = imp['cols'][-6:]
X.drop(columns=cols_to_drop, inplace = True)
X_train, X_test = split_vals(X, train_size)

In [123]:
m = RandomForestRegressor (n_estimators=1000, max_depth=None, min_samples_leaf = 3, max_features='auto', oob_score= True, random_state=1, n_jobs  = 3)
m.fit(X_train, y_train)
m.oob_score_
m.score(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=3, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=1000, n_jobs=3,
           oob_score=True, random_state=1, verbose=0, warm_start=False)

0.7377931623237937

0.9138686665807716

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [127]:

m2 = KNeighborsRegressor(n_neighbors=5, n_jobs  = 1)
m2.fit(X_train, y_train)
m2.score(X_train, y_train)

KNeighborsRegressor(algorithm='auto', leaf_size=30, metric='minkowski',
          metric_params=None, n_jobs=1, n_neighbors=5, p=2,
          weights='uniform')

0.47601271444342336

In [129]:
get_scores(m, X_train, X_test, y_train, y_test)

[2372.982170831708,
 4275.619459080817,
 0.9138686665807716,
 0.7238955204055199,
 0.7377931623237937]

In [131]:
pd.dt X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15576 entries, 11304 to 608
Data columns (total 10 columns):
road_name_code     15576 non-null int16
latitude           15576 non-null float64
longitude          15576 non-null float64
count_Year         15576 non-null int64
count_Month        15576 non-null int64
count_Week         15576 non-null int64
count_Day          15576 non-null int64
count_Dayofweek    15576 non-null int64
count_Dayofyear    15576 non-null int64
count_Elapsed      15576 non-null int64
dtypes: float64(2), int16(1), int64(7)
memory usage: 1.2 MB


In [ ]:
# build model and plot partial dependency plot to estimate the effects over time